#### Setup

In [ ]:
# import library
import tensorflow as tf
import numpy as np
import os
import time

In [ ]:
# Mendapatkan path file untuk dataset Shakespeare dari URL
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')


1115394/1115394 [==============================] - 0s 0us/step


#### Load Data

In [ ]:
# Membaca isi file dan mendekodekannya sebagai teks dengan encoding utf-8
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# Mencetak panjang teks dalam karakter
print(f'Length of text: {len(text)} characters')


Length of text: 1115394 characters


In [ ]:
# Mencetak 250 karakter pertama dari teks
print(text[:250])


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [ ]:
# Membuat kamus dari karakter unik dalam teks dan mengurutkannya
vocab = sorted(set(text))

# Mencetak jumlah karakter unik dalam teks
print(f'{len(vocab)} unique characters')


65 unique characters


#### Olah Teks

Vectorize Teks

In [ ]:
# Memecah teks contoh menjadi karakter-karakter menggunakan Unicode split
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')

# Menampilkan hasil pemecahan karakter
chars


<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
# Membuat layer StringLookup untuk mengonversi karakter menjadi ID
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab),
    mask_token=None
)


In [ ]:
# Menggunakan layer StringLookup untuk mengonversi karakter menjadi ID
ids = ids_from_chars(chars)

# Menampilkan hasil konversi
ids


<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [ ]:
# Membuat layer StringLookup untuk mengonversi ID kembali menjadi karakter
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(),
    invert=True,
    mask_token=None
)


In [ ]:
# Menggunakan layer StringLookup untuk mengonversi ID kembali menjadi karakter
chars = chars_from_ids(ids)

# Menampilkan hasil konversi
chars


<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
# Menggunakan tf.strings.reduce_join untuk menggabungkan karakter-karakter kembali menjadi teks
tf.strings.reduce_join(chars, axis=-1).numpy()


array([b'abcdefg', b'xyz'], dtype=object)

In [ ]:
# Mendefinisikan fungsi untuk mengonversi ID kembali menjadi teks
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)


Prediksi

In [ ]:
# Menggunakan layer StringLookup untuk mengonversi semua karakter dalam teks menjadi ID
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))

# Menampilkan hasil konversi
all_ids


<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [ ]:
# Membuat dataset dari tensor ID
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)


In [ ]:
# Menampilkan karakter-karakter pertama dalam 10 tensor ID
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [ ]:
# Menentukan panjang urutan yang diinginkan
seq_length = 100

In [ ]:
# Membuat urutan dari dataset dengan panjang yang ditentukan
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

# Menampilkan urutan karakter pertama dalam satu batch
for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [ ]:
# Menampilkan teks dari lima urutan pertama
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [ ]:
# Mendefinisikan fungsi untuk membagi input dan target dari suatu urutan
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text


In [ ]:
# Memanggil fungsi pada contoh urutan
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:

# Membuat dataset dari urutan dengan input dan target yang terpisah
dataset = sequences.map(split_input_target)

In [ ]:
# Menampilkan contoh input dan target dari dataset
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


Membuat Batch Training

In [ ]:
# Ukuran batch
BATCH_SIZE = 64

# Ukuran buffer untuk mengacak dataset
# (TF data dirancang untuk bekerja dengan urutan yang mungkin tak terbatas,
# sehingga tidak mencoba untuk mengacak seluruh urutan dalam memori. Sebaliknya,
# ia menjaga buffer di mana ia mengacak elemen).
BUFFER_SIZE = 10000

# Membuat dataset dengan mengacak, mengelompokkan, dan memuat dengan prefetch
dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

# Menampilkan dataset
dataset


<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

#### Buat Model

In [ ]:
# Panjang kamus dalam layer StringLookup
vocab_size = len(ids_from_chars.get_vocabulary())

# Dimensi embedding
embedding_dim = 256

# Jumlah unit RNN
rnn_units = 1024

In [ ]:
# Mendefinisikan kelas model yang merupakan turunan dari tf.keras.Model
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    # Lapisan embedding untuk mengonversi ID menjadi vektor embedding
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    # Lapisan GRU (Gated Recurrent Unit) untuk pemrosesan urutan
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    # Lapisan Dense untuk menghasilkan output final
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    # Melakukan embedding pada input
    x = self.embedding(x, training=training)
    # Mendapatkan nilai awal state GRU jika tidak ada
    if states is None:
      states = self.gru.get_initial_state(x)
    # Melakukan proses GRU pada urutan input
    x, states = self.gru(x, initial_state=states, training=training)
    # Melakukan lapisan Dense pada hasil GRU
    x = self.dense(x, training=training)

    # Mengembalikan hasil dan state jika diperlukan
    if return_state:
      return x, states
    else:
      return x

In [ ]:
# Membuat objek model menggunakan kelas yang telah didefinisikan
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

#### Uji Model

In [ ]:
# Melakukan prediksi pada satu batch contoh input dari dataset
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [ ]:
# Menampilkan ringkasan arsitektur model
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Menghasilkan indeks teracak dari distribusi prediksi
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [ ]:
sampled_indices

array([51, 16, 42,  2,  5,  7, 39,  9, 58, 52, 64, 50,  6, 42, 50, 51,  8,
       15, 35,  9, 61, 33, 15, 42, 45, 36, 41, 23,  4, 30, 42, 16, 54, 30,
       32, 26, 36, 25,  7, 32, 54, 44, 43, 52, 63, 58,  3, 50, 22, 20, 39,
       20, 37, 42, 44,  1, 43, 16, 43, 32, 40, 22, 43, 57, 13,  9, 41, 57,
       23, 14, 39,  7, 15, 21, 22, 11, 41,  8, 38, 65,  7, 20, 63, 56, 56,
        8, 37,  3, 15, 16, 63, 47, 64, 44, 61, 45, 25, 48, 29, 21])

In [ ]:
# Menampilkan contoh input dan prediksi karakter berikutnya
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'irror\nWhich shows me mine changed too; for I must be\nA party in this alteration, finding\nMyself thus'

Next Char Predictions:
 b"lCc &,Z.smyk'ckl-BV.vTBcfWbJ$QcCoQSMWL,Soedmxs!kIGZGXce\ndCdSaIdr?.brJAZ,BHI:b-Yz,Gxqq-X!BCxhyevfLiPH"


#### Train Model

Menambahkan optimizer dan fungsi loss

In [ ]:
# Menggunakan SparseCategoricalCrossentropy sebagai fungsi loss
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)


In [ ]:
# Menghitung rata-rata loss pada contoh batch
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.190125, shape=(), dtype=float32)


In [ ]:
# Menghitung eksp dari rata-rata loss
tf.exp(example_batch_mean_loss).numpy()

66.031044

In [ ]:
# Mengompilasi model dengan optimizer Adam dan fungsi loss SparseCategoricalCrossentropy
model.compile(optimizer='adam', loss=loss)

Konfigurasi Checkpoints

In [ ]:
# Direktori tempat checkpoint akan disimpan
checkpoint_dir = './training_checkpoints'
# Nama file checkpoint
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

# Callback untuk menyimpan checkpoint
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


Melakukan Proses Training

In [37]:
# Jumlah epoch
EPOCHS = 20

In [38]:
# Melatih model dengan dataset dan menyimpan checkpoint
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 15s 61ms/step - loss: 2.7271
Epoch 2/20
172/172 [==============================] - 12s 61ms/step - loss: 1.9869
Epoch 3/20
172/172 [==============================] - 12s 61ms/step - loss: 1.7092
Epoch 4/20
172/172 [==============================] - 14s 62ms/step - loss: 1.5500
Epoch 5/20
172/172 [==============================] - 12s 62ms/step - loss: 1.4527
Epoch 6/20
172/172 [==============================] - 13s 62ms/step - loss: 1.3854
Epoch 7/20
172/172 [==============================] - 12s 62ms/step - loss: 1.3341
Epoch 8/20
172/172 [==============================] - 12s 62ms/step - loss: 1.2899
Epoch 9/20
172/172 [==============================] - 12s 62ms/step - loss: 1.2486
Epoch 10/20
172/172 [==============================] - 15s 61ms/step - loss: 1.2099
Epoch 11/20
172/172 [==============================] - 14s 63ms/step - loss: 1.1697
Epoch 12/20
172/172 [==============================] - 12s 61ms/step - loss: 1.1288
E

#### Generate Teks

In [39]:
# Mendefinisikan kelas untuk menghasilkan teks satu langkah pada waktu inferensi
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Membuat mask untuk mencegah "[UNK]" dari dihasilkan.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Menempatkan -inf pada setiap indeks yang buruk.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Sesuaikan bentuk dengan vokabulari
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Mengonversi string ke token ID.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Menjalankan model.
    # predicted_logits.shape adalah [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Hanya menggunakan prediksi terakhir.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Menerapkan mask prediksi: mencegah "[UNK]" dari dihasilkan.
    predicted_logits = predicted_logits + self.prediction_mask

    # Mengambil sampel dari logits output untuk menghasilkan token ID.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Mengonversi dari token ID menjadi karakter
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Mengembalikan karakter dan state model.
    return predicted_chars, states

In [40]:
# Membuat objek model untuk inferensi satu langkah waktu
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [41]:
# Menjalankan inferensi pada satu contoh teks awal
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)


ROMEO:
Spakes: upon pala head o' the streets,
And every thing is here with chronators?
But widow Dido' a ballad applars that late heart's o'ly finger?
All must our horsion has a treacherous land Isabel,
I am too good for thee:--GONZALO:
I'll cry, God sleep wid is not comes.

SLY:
No, not in living adist: O, the news
I have a torgued father, die my back.

VIRGILIA:
None else, we stand alone, see another seck
Of the parties of certain dropsinds.

ROMEO:
We prove honest win that all abroad old steel,
With rained harbon his plother's love, the beauty was repair,
being but two scrupul of those hateful accest
how upon his country: this is sick for adile;
His adory Katharina to make a life
And holy charges me which indeed ere I chan a three.

Second Citizen:
We have fought with unterroplested language!
That he did ne'er revenge for, as she would say.
3 KING HENRY VI

QUEEN ELIZABETH:
Thus far as load as he was married any Juliet.

RIVERS:
O, wrapp'd and worldly thou hast emproving me?
No, no 

In [42]:
# Menjalankan inferensi pada beberapa contoh teks awal
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nThese tempted men that thou hast spoke, world's rage\nUpon the single sous wife any harm.\n\nLADY CAPULET:\nThe chaplain of the Martal'st December man\nI say, and scarr'd the delight walls;\nAnd by and by: this is the quarrel that I did: it\nhath best said, sir, if it be best one--I can be shere,\nWish warmous is deward him in the tooth.\n\nHASTINGS:\nI thank you, on my woman away a letter, thou\nshalt resinus? what end what heavens! Who's there?\n\nThird Citizen:\nTush, your amilias mould upon them,--\n\nMENENIUS:\nAs Surran, Percy, and by tribunes.\n\nSICINIUS:\nThe fighting\nHer she to sundering on the store,\nWho tamest on common nature, as he does.\n\nMARIANA:\nGood my liege, my messenger,\nI know not what? I think you are in last?\n\nKING EDWARD IV:\nWhy stoop not 'gainst all measure us,\nThe glorious world will rushe your cousins. This,\nLet it not so, farewell. What, with\nthe unlief and faint--'Side is changed to speak.\nWear me, marry, that I am her away

#### Ekspor Model Generator

In [43]:
# Menyimpan model satu langkah waktu sebagai SavedModel
tf.saved_model.save(one_step_model, 'one_step')

# Memuat model kembali
one_step_reloaded = tf.saved_model.load('one_step')


In [44]:
# Menjalankan inferensi pada model yang dimuat kembali
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

# Mencetak teks hasil inferensi
print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
Clarater Slain no less; but with this new grass how
He could gurst they bally doth beg the servants
